# Agent Short-Term Memory

Short-term memory stores conversation history within a session using checkpointers.

**What you'll learn:**
- Checkpointers persist conversation history
- SQLite for development, PostgreSQL for production
- Thread IDs manage separate sessions
- Access agent state in tools with ToolRuntime
- Modify agent state from tools for context offloading
- Save/load conversation summaries for long conversations

## Checkpointer Comparison

| Type | Use Case | Setup |
|------|----------|-------|
| **SQLite** | Development, testing | Simple file-based |
| **PostgreSQL** | Production, multi-user | Database connection |

In [ ]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage, ToolMessage, SystemMessage
from scripts import base_tools

from langchain.tools import tool, ToolRuntime
from langgraph.types import Command

from pydantic import BaseModel
from pathlib import Path

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

system_prompt = """You are a helpful assistant with memory.
- Remember previous messages in the conversation
- Use conversation history when answering questions
- Be concise and accurate"""

## Problem: No Memory

In [4]:
agent = create_agent(model=model, system_prompt=system_prompt)

agent.invoke({'messages': [HumanMessage("My name is Laxmi Kant")]})
response = agent.invoke({'messages': [HumanMessage("What's my name?")]})

response

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='9b296f74-456f-424c-9da6-dd25a9a1650d'),
  AIMessage(content="I don't know your name. You haven't told me yet!", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb809-fa04-7243-a3e7-8d484255bc4b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 85, 'total_tokens': 122, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 69}})]}

## Short-Term Memory: SQLite

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

os.makedirs("db", exist_ok=True)

conn = sqlite3.connect("db/checkpoints.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)
checkpointer.setup()

agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    checkpointer=checkpointer,
    system_prompt=system_prompt
)

In [6]:
config = {"configurable": {"thread_id": "user_123"}}

agent.invoke({"messages": [HumanMessage("My name is Laxmi Kant")]}, config)
response = agent.invoke({"messages": [HumanMessage("What's my name?")]}, config)

response

{'messages': [HumanMessage(content='My name is Laxmi Kant', additional_kwargs={}, response_metadata={}, id='f145ba2b-9597-4b9e-b631-8832e3efb1d2'),
  AIMessage(content='Hello Laxmi Kant! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb805-db78-73e0-8ea4-4a6320b5f569-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 238, 'output_tokens': 12, 'total_tokens': 250, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='796b91b6-3190-42fa-a89a-ccd91311b10f'),
  AIMessage(content='Your name is Laxmi Kant.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb805-e018-79d3-8bcd-1569560e921e-0', tool_calls=[], invalid_too

In [7]:
response

{'messages': [HumanMessage(content='My name is Laxmi Kant', additional_kwargs={}, response_metadata={}, id='f145ba2b-9597-4b9e-b631-8832e3efb1d2'),
  AIMessage(content='Hello Laxmi Kant! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb805-db78-73e0-8ea4-4a6320b5f569-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 238, 'output_tokens': 12, 'total_tokens': 250, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='796b91b6-3190-42fa-a89a-ccd91311b10f'),
  AIMessage(content='Your name is Laxmi Kant.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb805-e018-79d3-8bcd-1569560e921e-0', tool_calls=[], invalid_too

## Short-Term Memory: PostgreSQL

In [8]:
from langgraph.checkpoint.postgres import PostgresSaver
import psycopg

pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True)
pg_checkpointer = PostgresSaver(pg_conn)
pg_checkpointer.setup()

agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    checkpointer=pg_checkpointer,
    system_prompt=system_prompt
)

config = {"configurable": {"thread_id": "postgres_session"}}

agent.invoke({"messages": [HumanMessage("My city is Mumbai")]}, config)
response = agent.invoke({"messages": [HumanMessage("What's my city?")]}, config)

response['messages'][-1].content

'Your city is Mumbai.'

## Context Offloading: Read State in Tools

Access agent state to save conversation summaries for context management.

In [ ]:
from dataclasses import dataclass

# context is immutable
@dataclass
class UserContext:
    user_id: str
    session_id: str

@tool
def save_conversation_summary(summary: str, runtime: ToolRuntime):
    """Save conversation summary to disk for context offloading."""
    user_id = runtime.context.user_id
    thread_id = runtime.context.thread_id
    
    # Create directory structure
    summary_dir = Path(f"data/{user_id}/{thread_id}")
    summary_dir.mkdir(parents=True, exist_ok=True)
    
    # Write summary
    summary_path = summary_dir / "summary.md"
    summary_path.write_text(summary)
    
    return f"Summary saved to {summary_path}"

In [ ]:
# Test save summary
agent = create_agent(
    model=model,
    tools=[save_conversation_summary],
    checkpointer=checkpointer,
    context_schema=UserContext
)

config = {"configurable": {"thread_id": "session_1"}}
user_context = UserContext(user_id='kgptalkie', session_id='session_1')

response = agent.invoke({
    "messages": [HumanMessage("Save this summary: User discussed Python and AI topics")]
}, config=config, context=user_context)

response['messages'][-1].content

'I have saved the summary of our conversation.'

## Context Offloading: Modify State in Tools

Load previous summaries and inject them into agent state.

In [ ]:
@tool
def load_conversation_summary(runtime: ToolRuntime):
    """Load previous conversation summary from disk."""
    user_id = runtime.context.user_id
    # thread_id = runtime.context.thread_id
    thread_id = runtime.config['configurable']['thread_id']
    
    summary_path = Path(f"data/{user_id}/{thread_id}/summary.md")
    
    if not summary_path.exists():
        return Command(update={
            "messages": [
                ToolMessage(
                    "No previous summary found.",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        })
    
    # Read summary
    summary_text = summary_path.read_text()
    
    # Update state with summary as system message
    return Command(update={
        "messages": [
            SystemMessage(f"Previous conversation summary:\n{summary_text}"),
            ToolMessage(
                "Successfully loaded previous summary.",
                tool_call_id=runtime.tool_call_id
            )
        ]
    })

In [ ]:
# Test load summary
agent = create_agent(
    model=model,
    tools=[load_conversation_summary],
    checkpointer=checkpointer,
    context_schema=UserContext
)

response = agent.invoke({
    "messages": [HumanMessage("Load my previous conversation summary")]
}, config=config, context=user_context)

In [13]:
response

{'messages': [HumanMessage(content='Save this summary: User discussed Python and AI topics', additional_kwargs={}, response_metadata={}, id='55944055-3c8e-4800-9473-9fc5c9ff7d61'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'save_conversation_summary', 'arguments': '{"summary": "User discussed Python and AI topics"}'}, '__gemini_function_call_thought_signatures__': {'a3d81d05-605f-47e7-a5e1-a3a6c363ce00': 'CqkCAXLI2nyIaVdC0uFphiF6IaRSAzkKHz2ffHXDdxm1ahOYK2NBBm4KdwHanHq/sBr78FSivq7Ki67tfGiMhzzSyc039FZtvIut6bOD1McYw457m27yMPID47UyEa4NsiMACCPS2YZcnTOisYUIZidGbium5pyMLieBkehAM0Ust7WNvRN0bk2f+JECFNuWYxTVW3ehU7OV/S8uM7so2a5LjijuFC9UEX2+komNo09xZyW8xscp3r/Vs2AnK5gQ0vmzuEcSLG1Cd4pU0kbSiEIvwQSBS6V7+rojxhu26S1abdLiuV4ssXZWq7dfnFWpu1PhCN/Yv8lHBwBJJYN6czDPgSLos2GNCtxqgkV6D8Kp9kpmahAm2TIaHzwH6CFu/W5HjQS7i926rcoK'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bb805-ff4

## Combined: Save and Load Context

Real-world example combining both tools for context management.

In [ ]:
# Agent with both save and load
agent = create_agent(
    model=model,
    tools=[save_conversation_summary, load_conversation_summary],
    checkpointer=checkpointer,
    context_schema=UserContext
)

# Session 2: Have conversation and save summary
config = {"configurable": {"thread_id": "session_2"}}
user_context = UserContext(user_id='kgptalkie', session_id='session_2')

agent.invoke({
    "messages": [HumanMessage("We're building a chatbot using LangChain")]
}, config=config, context=user_context)

agent.invoke({
    "messages": [HumanMessage("Save summary: Building LangChain chatbot with memory and tools")]
}, config=config, context=user_context)

print("Summary saved for project_alpha")

Summary saved for project_alpha
